Setup (general)

In [1]:
%load_ext autoreload
%autoreload 2

Load the toolkit, setup input data


In [2]:
import spikeextractors as se
import spiketoolkit as st
import spikesorters as ss

# filepath = 'G:/Data_julien/Klusta_results/Barrel1/Barrel1.dat'
filepath = 'C:/tmp/TT_Hpc5/TT_Hpc5.dat'
rec = se.BinDatRecordingExtractor(filepath,
                                  numchan=4,
                                  dtype='int16',
                                  sampling_frequency=32000)

rec = rec.load_probe_file('C:/tmp/TT_Hpc5/TT_Hpc5.prb')
# rec = rec.load_probe_file('G:/Data_julien/Klusta_results/Barrel1/Barrel1.prb')

14:55:17 [I] klustakwik KlustaKwik2 version 0.2.6


In [3]:
print(ss.available_sorters())
print(ss.installed_sorter_list)
print(ss.get_default_params('klusta'))

klusta_params = ss.get_default_params('klusta')
# sorting_klusta = ss.run_klusta(recording=rec,
#                                verbose=True,
#                                **klusta_params)

['hdsort', 'herdingspikes', 'ironclust', 'kilosort', 'kilosort2', 'klusta', 'mountainsort4', 'spykingcircus', 'tridesclous', 'waveclus']
[<class 'spikesorters.klusta.klusta.KlustaSorter'>]
{'adjacency_radius': None, 'threshold_strong_std_factor': 5, 'threshold_weak_std_factor': 2, 'detect_sign': -1, 'extract_s_before': 16, 'extract_s_after': 32, 'n_features_per_channel': 3, 'pca_n_waveforms_max': 10000, 'num_starting_clusters': 50}


In [4]:
# st.postprocessing.export_to_phy(
#     recording=rec,
#     sorting=sorting_klusta,
#     output_folder='G:/Data_julien/Klusta_results/Barrel1/phy'
# )

In [5]:
# sor = se.PhySortingExtractor(
#     'G:/Data_julien/Klusta_results/Barrel1/phy'
# )

sor = se.KlustaSortingExtractor(file_or_folder_path='C:/tmp/TT_Hpc5')
uids = sor.get_unit_ids()
good_units = [uid for uid in uids if sor.get_unit_property(uid, 'quality') == 'good']


res = st.postprocessing.compute_waveform_features(
    sorting=sor,
    recording=rec,
    as_dataframe=True,
    save_property_or_features=True,
    upsampling_factor=4,
)



c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_stats_mstats_common.py:140: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\thijs\miniconda3\envs\spiketoolkit\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\thijs\miniconda3\envs\spiketoolkit\lib

In [11]:
from spiketoolkit.postprocessing.plot_features import plot_metrics

for uid in good_units:
    plot_metrics(
        sor,
        rec,
        uid,
        invert_template=True,
        upsampling_factor=4,
        )